Для начала необходимо собрать датасет с текстами песен. В моей работе класстерами будут  исполнители. Собирать данные для датасета буду с помощью парсера с сайта https://www.litprichal.ru/ (сам парсер можно посмотреть в репозитории, файл parser.py), результат работы парсера буду ложить в файл songs.csv. При этом формат каждой записи такой: Исполнитель, Название песни, Текст. 

Данные уже сразу отфильтрованы: переведены в нижний регистр, оставлены только русские символы (так как и исполнители русские), удалены знаки препинания, переноса и т.д. То есть текст песен в формате:  ("слово1 слово2 слово3 ...") 

Изначальные класстеры - 'Король и шут', 'Владимир Высоцкий', 'Смешарики', 'ДДТ', 'Юрий Шатунов',  'Руки вверх', 'Виктор Цой.


Размер датасета - 740 записей'

Импортируем все необходимые модули в блоке ниже

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.cluster import KMeans #https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans

Открываю файл "songs.csv" и записываю данные в dataframe pandas (кодировка windows-1251, так как текст русский)

Выведу датафрейм, чтобы показать существование данных

In [15]:
data = pd.read_csv("songs.csv", encoding='windows-1251')
data

,Исполнитель,Название песни,Текст
0,Король и шут,некромант ты говоришь я демон так и есть со мн...,ты говоришь я демон так и есть со мною не вида...
1,Король и шут,мертвый анархист,ослепший старый маг ночью по лесу бродил на кл...
2,Король и шут,кукла колдуна акустика,темный мрачный коридор я на цыпочках как вор п...
3,Король и шут,лесник,замученный дорогой я выбился из сил и в доме л...
4,Король и шут,прыгну со скалы,с головы сорвал ветер мой колпак я хотел лю...
...,...,...,...
735,Виктор Цой,алюминевые огурцы,здравствуйте девочки здравствуйте мальчики смо...
736,Виктор Цой,дальше действовать будем мы от паши корейца и,мы хотим видеть дальше чем окна дома напротив ...
737,Виктор Цой,следи за собой,сегодня кому то говорят до свиданья завтра ска...
738,Виктор Цой,перемен от паши корейца и,вместо тепла зелень стекла вместо огня дым из ...


Удаляем столбец с названием песен и перемешиваем строки, так как сейчас они идут группами по порядку исполнителей.

In [16]:
data = data.drop('Название песни', axis=1)
data = data.sample(frac=1).reset_index(drop=True)
data

,Исполнитель,Текст
0,Король и шут,во мрак погружен туманный альбион в долине пок...
1,ДДТ,ночь куранты бьются злея чтото брат им невдоме...
2,Король и шут,какой таинственной казалась мне та ночь я зату...
3,Владимир Высоцкий,зарыты в нашу память на века и даты и события ...
4,Руки вверх,до чего ж я невезучий до чего ж я невезучий та...
...,...,...
735,Король и шут,как юнец сто ночей шел за ней я шел за ней мне...
736,Юрий Шатунов,закат окончил летний теплый вечер остановился ...
737,ДДТ,это было в ленинграде и довольно давно на свет...
738,Юрий Шатунов,мы сегодня с тобою вдвоем не хочу ни о чем гов...


Далее с помощью LabelEncoder переведу названия исполнителей в численный вид. Создам переменную Y_true, чтобы в последующем было удобнее работать с метриками расчета качества кластеризации

In [17]:
encoder = LabelEncoder()
data['Исполнитель'] = encoder.fit_transform(data['Исполнитель'])
Y_true = np.array(data['Исполнитель'])

Попробую сначала обработать тексты с помощью LabelEncoder, а затем с помощью KMeans провести класстеризацию и посмотреть качество с помощью метрик:

Adjusted Rand Index (ARI): Вычисляет сходство между двумя наборами меток, предоставляя случайную оценку, если метки равномерно распределены.

Homogeneity Score: Измеряет, насколько каждый кластер состоит из только одного класса.

Completeness Score: Измеряет, насколько все экземпляры одного класса относятся к одному и тому же кластеру.

Silhouette Coefficient: Измеряет среднее расстояние между объектами внутри кластера и объектами в соседних кластерах. Значение должно быть ближе к 1 для хорошей кластеризации.

In [45]:
def show_metrics(Y_true, Y_pred):
    ari_score = metrics.adjusted_rand_score(Y_true, Y_pred)
    print("Adjusted Rand Index (ARI): ", ari_score)
    homogeneity_score = metrics.homogeneity_score(Y_true, Y_pred)
    print("Homogeneity Score: ", homogeneity_score)
    completeness_score = metrics.completeness_score(Y_true, Y_pred)
    print("Completeness Score: ", completeness_score)
    silhouette_coefficient = metrics.silhouette_score(Y_true.reshape(-1, 1), Y_pred)
    print("Silhouette Coefficient: ", silhouette_coefficient)
    

X = encoder.fit_transform(data['Текст'])
X = X.reshape(-1, 1)

kmeans = KMeans(n_clusters=data['Исполнитель'].max()+1, n_init='auto' )
kmeans.fit(X)
Y_pred = kmeans.predict(X)

show_metrics(Y_true, Y_pred)

Adjusted Rand Index (ARI):  0.01064275610737603
Homogeneity Score:  0.031115344206576082
Completeness Score:  0.028809945575825606
Silhouette Coefficient:  -0.08239928600393147


Теперь запустим кластеризацию с теми же данными, но с помощью MiniBatchKMeans (интересно сравнить на сколько будет разница)

In [56]:
minibatch_kmeans = MiniBatchKMeans(n_clusters=data['Исполнитель'].max()+1, n_init='auto')
minibatch_kmeans.fit(X)
Y_pred = minibatch_kmeans.predict(X)

show_metrics(Y_true, Y_pred)

Adjusted Rand Index (ARI):  0.010926583072114481
Homogeneity Score:  0.03362144215189391
Completeness Score:  0.031195532833832383
Silhouette Coefficient:  -0.0947927401509405


Результат кластеризации пока что очень плохой, мы можно сказать рандомно разделили классы. Про сравнение эффективности MiniBatchKMeans и KMeans сказать сложно, так как результаты плохие у обоих алгоритмов

Попробую улучшить результат класстеризации с помощью использования TF-IDF вместо LabelEncoder. Посмотрим, что из этого выйдет, но результат должен стать лучше

Сначала также использую KMeans

In [62]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Текст'])

kmeans = KMeans(n_clusters=data['Исполнитель'].max()+1, n_init='auto' )
kmeans.fit(X)
Y_pred = kmeans.predict(X)

show_metrics(Y_true, Y_pred)

Adjusted Rand Index (ARI):  0.12743366836508013
Homogeneity Score:  0.16198418841247425
Completeness Score:  0.16339706229343431
Silhouette Coefficient:  -0.12151119201011261


Теперь запустим кластеризацию с теми же данными, но с помощью MiniBatchKMeans

In [69]:
minibatch_kmeans = MiniBatchKMeans(n_clusters=data['Исполнитель'].max()+1, n_init='auto')
minibatch_kmeans.fit(X)
Y_pred = minibatch_kmeans.predict(X)

show_metrics(Y_true, Y_pred)

Adjusted Rand Index (ARI):  0.1250929016200852
Homogeneity Score:  0.1703927943170387
Completeness Score:  0.2347372418933311
Silhouette Coefficient:  -0.2620462451276643


Как можно увидеть по метрикам, результат стал лучше по сравнению с предыдущей итерацией, но в целом остался все таким же низким (или около рандомным). При этом MiniBatchKMeans отрабатывает немного лучше, чем обычный KMeans 

<b>Вывод</b>: разделять песни по исполнителям только на основе текста – плохая идея, так как классификация в этом случае не дает приемлимого результата. Это связано с тем, что на основе только слов сложно определить конкретного исполнителя (к примеру в ДЗ №2 мы могли разделять имена, так как в именах есть определенная зависимость между полом и именем, а тут такой нет). Возможно, стоило бы использовать не только слова песни, а также дополнительно отцифровку звука исполнения конкретным исполнителем, амплитуду звука, продолжительночть песни и другие возможные параметры